In [ ]:
PATH = "/home/xjaros2/Documents/git/csidh-setup/"
SCOPETYPE = 'OPENADC'
PLATFORM = 'CW308_STM32F3'

SS_VER = 'SS_VER_2_1'
attack_type = "A1"

In [ ]:
# These need to be measured manually, will be used in the figures
# Also, bit of cheating here, since this means we know what degree isogeny is where
dummy_max_ext_offset = 4705896
dummy_isogeny_data = [
    {"degree": 3, "setup_start": 0,      "isogeny_start": 267412,  "isogeny_end": 327688},
    {"degree": 5, "setup_start": 327752, "isogeny_start": 447160,  "isogeny_end": 531468},
    {"degree": 7, "setup_start": 531500, "isogeny_start": 619624,  "isogeny_end": 675316},
    {"degree": 3, "setup_start": 675332, "isogeny_start": 1114596, "isogeny_end": 1175536},
    {"degree": 5, "setup_start": 1175588,"isogeny_start": 1295864, "isogeny_end": 1380000},
    {"degree": 7, "setup_start": 1380024,"isogeny_start": 1466320, "isogeny_end": 1523632},
    {"degree": 5, "setup_start": 1523672,"isogeny_start": 2083708, "isogeny_end": 2168156},
    {"degree": 3, "setup_start": 2168232,"isogeny_start": 2694412, "isogeny_end": 2755560},
    {"degree": 7, "setup_start": 2755604,"isogeny_start": 2962932, "isogeny_end": 3020256},
    {"degree": 3, "setup_start": 3020396,"isogeny_start": 3457520, "isogeny_end": 3518488},
    {"degree": 5, "setup_start": 3518528,"isogeny_start": 3638024, "isogeny_end": 3729888},
    {"degree": 7, "setup_start": 3730068,"isogeny_start": 3815672, "isogeny_end": 3873328},
    {"degree": 3, "setup_start": 3873524,"isogeny_start": 4214548, "isogeny_end": 4275180},
    {"degree": 5, "setup_start": 4275208,"isogeny_start": 4395016, "isogeny_end": 4486364},
    {"degree": 7, "setup_start": 4486600,"isogeny_start": 4573456, "isogeny_end": 4631104},
]

In [ ]:
from csidhtools.search.io import read_caches_into_dataframe
%cd $PATH
husky = [
    "./notebooks/husky-clock-isogeny-single-glitches-all-150k.json",
    "./notebooks/husky-clock-isogeny-single-glitches-chosen-150k.json",
    "./notebooks/husky-clock-isogeny-single-glitches-three-150k.json"
]

df = read_caches_into_dataframe(husky)
df

In [ ]:
from csidhtools import CSIDHDLL
PATH = "/home/xjaros2/Documents/git/csidh-setup/csidh-target/src/"
csidhdll = CSIDHDLL(src_path=PATH)

In [ ]:
ISOGENY_NEIGHBOURS = \
    csidhdll.isogeny_neighbors(158, 0) + \
    csidhdll.isogeny_neighbors(158, 1) + \
    csidhdll.isogeny_neighbors(158, 2)
ISOGENY_NEIGHBOURS

In [ ]:
POTENTIALLY_SKIPPED_ISOGENY = df.iloc[[
    i for i, x in enumerate(df["responses"]) if any(y in x for y in ISOGENY_NEIGHBOURS)]
]
POTENTIALLY_SKIPPED_ISOGENY.responses = POTENTIALLY_SKIPPED_ISOGENY.responses.apply(lambda x: x[0])
POTENTIALLY_SKIPPED_ISOGENY

In [ ]:
PATH = "/home/xjaros2/Documents/git/csidh-setup/"
%cd $PATH/notebooks
%run ./figures/single-glitch-isogeny-figure.ipynb
plot_isogeny_offsets(
    potential_skips=POTENTIALLY_SKIPPED_ISOGENY,
    isogeny_data=dummy_isogeny_data,
    label_map= {
        "0": "($+$) $3$-isogeny", 
        "410": "($-$) $3$-isogeny",
        "295": "($+$) $5$-isogeny",
        "390": "($-$) $5$-isogeny", 
        "15": "($+$) $7$-isogeny",
        "144": "($-$) $7$-isogeny"
    },
    yticks=ISOGENY_NEIGHBOURS,
    use_latex=True
)

In [ ]:
skips =  df[df["responses"].isin([[x] for x in ISOGENY_NEIGHBOURS])]
stats = {
    "Total": len(df),
    "Normal": len(df[df.type == "NORMAL"]),
    "Reset": len(df[df.type == "RESET"]),
    "Justright": len(df[df.type == "JUSTRIGHT"]),
    "(+)3" : len(df[df["responses"].isin([[0]])]),
    "(-)3" : len(df[df["responses"].isin([[410]])]),
    "(+)5" : len(df[df["responses"].isin([[295]])]),
    "(-)5" : len(df[df["responses"].isin([[390]])]),
    "(+)7" : len(df[df["responses"].isin([[15]])]),
    "(-)7" : len(df[df["responses"].isin([[144]])]),
    "skipped": len(skips)
}
percentages = {}
for key in stats:
    percentages[key] = 100 * stats[key]/stats["Total"]
percentages, stats

In [ ]:
single_skips = list(set(potential_skips["unit"]))

In [ ]:
%cd $PATH/notebooks
%run ./init.ipynb
%cd $PATH/notebooks
# csidh.voltage_glitching_setup()
csidh.scope.cglitch_setup()
csidh.action_sleep = 0.25
%cd $PATH/notebooks

In [ ]:
csidh.reset_target()
csidh.scope.arm()
csidh.action()
ret = csidh.scope.capture()
if ret:
    print("Timeout happened during acquisition")
PUBLIC_EXPECTED = csidh.public_with_errors
MAX_EXT_OFFSET = csidh.scope.adc.trig_count

print("#" * 80)
print(f"PUBLIC: {PUBLIC_EXPECTED}")
print(f"TRIGS:  {MAX_EXT_OFFSET}")
print("#" * 80)

In [ ]:
csidh.scope.glitch.num_glitches = 2

In [ ]:
csidh.dis()

In [ ]:
# cache = OrderedDict()
# REPEATS = 20000
# cache_interval = 2
# for _ in tqdm(range(REPEATS//cache_interval)):
#     batch = []
#     for _ in range(cache_interval):
#         i = random.randint(0, len(single_skips)-1)
#         j = i
#         while j == i:
#             j = random.randint(0, len(single_skips)-1)

#         fst = single_skips[i]
#         snd = single_skips[j]
#         swap = lambda x, y: (y, x)
#         if fst.ext_offset > snd.ext_offset:
#             fst, snd = swap(fst, snd)
            
#         unit1 = Unit(repr=repr(fst))
#         unit1.repeat = [fst.repeat, fst.repeat]
#         unit1.ext_offset = [fst.ext_offset, snd.ext_offset-fst.ext_offset]

#         unit2 = Unit(repr=repr(snd))
#         unit2.repeat = [snd.repeat, snd.repeat]
#         unit2.ext_offset = [fst.ext_offset, snd.ext_offset-fst.ext_offset]
            
#         unit3 = Unit(repr=repr(fst))
#         unit3.repeat = [fst.repeat, snd.repeat]
#         unit3.ext_offset = [fst.ext_offset, snd.ext_offset-fst.ext_offset]

#         unit4 = Unit(repr=repr(snd))
#         unit4.repeat = [fst.repeat, snd.repeat]
#         unit4.ext_offset = [fst.ext_offset, snd.ext_offset-fst.ext_offset]

#         print(unit1)
#         print(unit2)
#         print(unit3)
#         print(unit4)
        
#         batch.append(unit1)
#         batch.append(unit2)       
#         batch.append(unit3)
#         batch.append(unit4)

#     evaluate_batch(csidh, batch)
#     write_cache_to_file(f"husky-clock-ISOGENY-DOUBLE-SKIP.json", cache, 1, len(cache), -1)   

In [ ]:
# potential = [
#     "./husky-clock-xISOG-first-5-isogeny-skip-search-until7.json"
# ]

# df = read_cachefiles_to_dataframe(potential)
# df = df[(df["type"] == "JUSTRIGHT")].reset_index()
# indices = [i  for i,x in enumerate(df["responses"]) if 199 in x]
# potential_skips = df.iloc[indices]
# batch = list(set(potential_skips["unit"]))
# batch.sort(key=lambda unit: -unit.ext_offset)
# batch
# cache = OrderedDict()
# uut = batch[-1]
# uut.offset = 2706
# uut.width = 2600
# uut.repeat = 4
# evaluate_batch(csidh, [uut])
# write_cache_to_file(f"first-5-isogeny-potential-skips.json", cache, 1, len(cache), -1)   

In [ ]:
# cache = OrderedDict()
# REPEATS = 20000
# cache_interval = 2
# for _ in tqdm(range(REPEATS//cache_interval)):
#     batch = []
#     for _ in range(cache_interval):
#         i = random.randint(0, len(candidates)-1)
#         base_unit = candidates[i]
#         for j in range(len(isogeny_offsets)):
#             if j == 0:
#                 eof_min = 0
#                 eof_max = isogeny_offsets[0][1]
#             else:
#                 eof_min = isogeny_offsets[j-1][2]
#                 eof_max = isogeny_offsets[j][1]
#             unit = Unit(repr=repr(base_unit))
#             unit.ext_offset = random.randint(eof_min, eof_max)
#             print(unit)
#             batch.append(unit)

#     evaluate_batch(csidh, batch)
#     write_cache_to_file(f"husky-clock-ISOGENY-SKIP-SEARCH.json", cache, 1, len(cache), -1)   

In [ ]:
# cache = OrderedDict()

# REPEATS = 17500
# cache_interval = 10
# for _ in tqdm(range(REPEATS//cache_interval)):
#     batch = []
#     for _ in range(cache_interval):
#         # width = random.randint(2650, 2750)
#         # offset = random.randint(2560, 2610)
#         # repeat = random.randint(4, 5)
#         width = random.randint(0, csidh.scope.glitch.phase_shift_steps//2)
#         offset = random.randint(0, csidh.scope.glitch.phase_shift_steps)
#         repeat = random.randint(1, 18)
        
#         ext_offset = random.randint((474252//4 )+2000, (474252//4 ) + 3200)
    
#         unit = Unit(repr=f"{ext_offset},{offset},{width},{repeat},None,0")
#         batch.append(unit)

#     evaluate_batch(csidh, batch)
#     write_cache_to_file(f"husky-clock-xISOG-first-5-isogeny-skip-parameter-search.json", cache, 1, len(cache), -1)   

In [ ]:
# population = []
# cache = OrderedDict()
# for i, (_, isogeny_offset) in tqdm(enumerate(isogeny_offsets[:1])):
#     Unit.is_husky = csidh.scope._is_husky
#     Unit.OFFSET_MIN = 2705 	 
#     Unit.OFFSET_MAX = 2705 	
#     Unit.OFFSET_RANGE = Unit.OFFSET_MAX - Unit.OFFSET_MIN
    
#     Unit.WIDTH_MIN = 2601
#     Unit.WIDTH_MAX = 2601
#     Unit.WIDTH_RANGE = Unit.WIDTH_MAX - Unit.WIDTH_MIN
    
#     Unit.EXT_OFFSET_MIN = isogeny_offset // 4 + 0
#     Unit.EXT_OFFSET_MAX = isogeny_offset // 4 + 10000
#     Unit.EXT_OFFSET_RANGE = Unit.EXT_OFFSET_MAX - Unit.EXT_OFFSET_MIN
    
#     Unit.REPEAT_MIN = 4
#     Unit.REPEAT_MAX = 4
#     Unit.repeat_range = Unit.REPEAT_MAX - Unit.REPEAT_MIN
#     batch = generate_whole_population()
#     print(f"Evaluating population for Isogeny number: {i}, {Unit.EXT_OFFSET_MIN=}, {Unit.EXT_OFFSET_MAX=}, Batch size: {len(batch)}")
#     population += batch
#     print(len(population))
#     evaluate_batch(csidh, batch)
#     write_cache_to_file(f"xISOG-5-isogeny-skip-test.json", cache, 1, len(population), -1)   